In [1]:
import os

os.chdir("../")
import numpy as np
import torch as th
from glob import glob
from utils import *
import ray
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from alphagen.config import HORIZON_BACK


In [2]:
json_paths = glob("checkpoints/*ret1d*")
json_paths = [
    sorted(
        glob(f"{name}/*.json"),
        key=lambda x: int(x.split("/")[-1].split("_")[0]),
    )[-1]
    for name in json_paths
]


In [3]:
data = ArgData(
    start_time=20190103,
    end_time=20211231,
    device=torch.device("cpu"),
    max_backtrack_days=HORIZON_BACK,
)


In [4]:
def json_to_factor(path):
    with open(path, "r") as f:
        alpha = json.load(f)
    factors = [
        Feature(getattr(FeatureType, expr[1:])).evaluate(data)
        if expr[0] == "$"
        else formula_to_expression(expr).evaluate(data)
        for expr in alpha["exprs"]
    ]
    weights = torch.tensor(alpha["weights"])
    factor_value = sum(f * w for f, w in zip(factors, weights))
    factor_value = normalize_by_day(factor_value)
    return factor_value


In [5]:
from tqdm import  trange
alphas =  []
for i in trange(len(json_paths)):
    alphas.append(json_to_factor(json_paths[i]))

 12%|███████▎                                                    | 4/33 [02:52<23:14, 48.08s/it]

In [ ]:
# from concurrent.futures import ProcessPoolExecutor

# alphas = []
# with ProcessPoolExecutor(80) as executor:
#     for value in zip(
#         executor.map(
#             json_to_factor,
#             json_paths,
#         ),
#     ):
#         alphas.append(value)

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 48248784000 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
sig_ret1d = th.stack([i[0] for i in alphas], dim=-1)


In [ ]:
sig_ret1d = th.concatenate(
    [th.zeros(HORIZON_BACK, 6000, len(alphas)), sig_ret1d], dim=0
)


In [ ]:
import importlib
import alphagen_ocean
importlib.reload(alphagen_ocean.calculator)

from alphagen_ocean.calculator import QLibStockDataCalculator


In [ ]:
cal = QLibStockDataCalculator(data)


In [ ]:
ret1d = cal.raw_ret1d
ret2d = cal.raw_ret2d
ret5d = cal.raw_ret5d


In [ ]:
len(ret1d)

11358

Exception ignored in atexit callback: <bound method InteractiveShell.atexit_operations of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f857efcba90>>
Traceback (most recent call last):
  File "/home/ray/anaconda3/envs/ag/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3875, in atexit_operations
    self._atexit_once()
  File "/home/ray/anaconda3/envs/ag/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3854, in _atexit_once
    self.reset(new_session=False)
  File "/home/ray/anaconda3/envs/ag/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 1397, in reset
    del ns[k]
        ~~^^^
  File "/home/ray/anaconda3/envs/ag/lib/python3.11/site-packages/ray/_private/worker.py", line 1723, in sigterm_handler
    sys.exit(signum)
SystemExit: 15


In [ ]:
ic = []
for i in range(sig_ret1d.shape[-1]):
    ic.append(batch_pearsonr(sig_ret1d[..., i], ret1d))


RuntimeError: The size of tensor a (11664) must match the size of tensor b (11358) at non-singleton dimension 0

In [ ]:
mean_ic = [th.mean(i) for i in ic]


In [ ]:
split_idx = 486 * 16


In [ ]:
demo = ic[0]
demo = np.array(demo)


In [ ]:
demo.shape

(11358,)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
reshaped_demo = demo.reshape(-1,16)
average_ic_16_points = np.mean(reshaped_demo, axis=0)

num_months = reshaped_demo.shape[0] // 21

monthly_ic = np.mean(reshaped_demo[:num_months * 21].reshape(num_months, 21, 16), axis=1).mean(axis=1)

padded_monthly_ic = np.pad(monthly_ic, (0, 36 - len(monthly_ic)), 'constant')
monthly_ic_reshaped = padded_monthly_ic.reshape(6, 6)
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
monthly_ic = np.mean(
    reshaped_demo[: num_months * 21].reshape(num_months, 21, 16), axis=1
).mean(axis=1)
plt.xlabel('Time Points')
padded_monthly_ic = np.pad(monthly_ic, (0, 36 - len(monthly_ic)), "constant")
plt.title('Average IC at 16 Time Points')

plt.subplot(1, 2, 2)
sns.heatmap(monthly_ic_reshaped, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Monthly IC'})
plt.xlabel('6-Month Period')
plt.ylabel('Year')
plt.xlabel("Time Points")
plt.ylabel("Average IC")
plt.title("Average IC at 16 Time Points")
plt.show()
sns.heatmap(
    monthly_ic_reshaped, annot=True, cmap="YlGnBu", cbar_kws={"label": "Monthly IC"}
)
plt.xlabel("6-Month Period")
plt.ylabel("Year")
plt.title("Monthly IC for 3 Years (6x6 Grid)")


ValueError: cannot reshape array of size 11358 into shape (16)